# Hex

## Import and setup

In [87]:
import math
from math import inf

# Define the board size for Hex
BOARD_SIZE = 4
DEPTH = 4

# Define player symbols
PLAYER_X = 'X'
PLAYER_O = 'O'
EMPTY = ' '
# X is vertical, O is horizontal

## Check if the board is full

In [88]:
def isfull(board):
    count = 0
    for i in range(BOARD_SIZE):
        for j in range(BOARD_SIZE):
            if board[i][j] == EMPTY:
                count += 1
    return count == 0

## Path finding and evaluation

There are several versions of this function (outdated ones are commented)

In [89]:
# def path_find(board, player, verbose=False):
#     # return 1 if the player has a path
#     fringe = []
#     visited = set()
#     # If player is the vertical
#     if player == PLAYER_X:
#         for index, symbol in enumerate(board[0]):
#             if symbol == PLAYER_X:
#                 fringe.append((0,index))
#         if verbose:
#             print(fringe)
#         while True:
#             # If we can no longer expand return 0
#             if len(fringe) == 0:
#                 if verbose:
#                     print(visited)
#                 return 0
#             top = fringe[0]
#             row,col = top
#             # If the level is at the other side
#             if row == BOARD_SIZE -1:
#                 if verbose:
#                     print(visited)
#                 return 1
#             # Adding the expanded element to the visited set
#             visited.add(top)
#             # Remove the expanded element from the list
#             fringe = fringe[1:]
#             # Add child nodes to the fringe
#             # If the node is left most, we start checking at the direct downword element
#             # If the node is not left most, we start checking at the element leftward of it
#             new_col = col if col == 0 else col-1
#             new_row = row + 1
#             while True:
#                 if board[new_row][new_col] == PLAYER_X:
#                     current = (new_row,new_col)
#                     if current not in visited:
#                         # if the node that is connected is never visited before push it onto the fringe
#                         fringe = [current] + fringe
#                 else:
#                     # check if the element is not the leftward element
#                     if new_col != col - 1: 
#                         break
#                 new_col += 1
#                 if new_col == BOARD_SIZE:
#                     break
#             if verbose:
#                 print(fringe)

#     # If player is the horizontal
#     elif player == PLAYER_O:
#         for index, symbol in enumerate([layer[0] for layer in board]):
#             if symbol == PLAYER_O:
#                 fringe.append((index,0))
#         if verbose:
#             print(fringe)
#         while True:
#             # If we can no longer expand return 0
#             if len(fringe) == 0:
#                 if verbose:
#                     print(visited)
#                 return 0
#             top = fringe[0]
#             row,col = top
#             # If the level is at the other side
#             if col == BOARD_SIZE -1:
#                 if verbose:
#                     print(visited)
#                 return 1
#             # Adding the expanded element to the visited set
#             visited.add(top)
#             # Remove the expanded element from the list
#             fringe = fringe[1:]
#             # Add child nodes to the fringe
#             # If the node is top most, we start checking at the direct rightward element
#             # If the node is not top most, we start checking at the element upward of it
#             new_row = row if row == 0 else row-1
#             new_col = col + 1
#             while True:
#                 if board[new_row][new_col] == PLAYER_O:
#                     current = (new_row,new_col)
#                     if current not in visited:
#                         # if the node that is connected is never visited before push it onto the fringe
#                         # print(current)
#                         fringe = [current] + fringe
#                 else:
#                     # check if the element is not the top most element
#                     if new_row != row - 1: 
#                         break
#                 new_row += 1
#                 if new_row == BOARD_SIZE:
#                     break
#             if verbose:
#                 print(fringe)

# def evaluate(board,verbose=False):
#     if verbose:
#         print("X:")
#     x = path_find(board,PLAYER_X,verbose)
#     if x == 1:
#         return 1
#     if verbose:
#         print("O:")
#     o = path_find(board,PLAYER_O,verbose)
#     if o == 1:
#         return -1
#     return 0


In [90]:
# def path_find(board, player, verbose=False):
#     # return 1 if the player has a path else return the fractional path
#     fringe = []
#     visited = set()
#     max_depth = 1
#     # If player is the vertical
#     if player == PLAYER_X:
#         for index, symbol in enumerate(board[0]):
#             if symbol == PLAYER_X:
#                 fringe.append((0,index,1))
#         if verbose:
#             print(fringe)
#         while True:
#             # If we can no longer expand return 0
#             if len(fringe) == 0:
#                 if verbose:
#                     # print("empty fringe")
#                     print(visited, max_depth, BOARD_SIZE)
#                 return max_depth/BOARD_SIZE
#             top = fringe[0]
#             row,col,depth = top
#             # If the level is at the other side
#             if row == BOARD_SIZE -1:
#                 if verbose:
#                     # print("reached")
#                     print(visited)
#                 return 1
#             # Adding the expanded element to the visited set
#             visited.add(top)
#             # Remove the expanded element from the list
#             fringe = fringe[1:]
#             # Add child nodes to the fringe
#             # If the node is left most, we start checking at the direct downword element
#             # If the node is not left most, we start checking at the element leftward of it
#             new_col = col if col == 0 else col-1
#             new_row = row + 1
#             new_depth = depth + 1
#             while True:
#                 if board[new_row][new_col] == PLAYER_X:
#                     current = (new_row,new_col,new_depth)
#                     if current not in visited:
#                         # if the node that is connected is never visited before push it onto the fringe
#                         fringe = [current] + fringe
#                         if new_depth > max_depth:
#                             max_depth = new_depth
#                 else:
#                     # check if the element is not the leftward element, if it ain't keep checking
#                     if new_col != col - 1: 
#                         break
#                 new_col += 1
#                 if new_col == BOARD_SIZE:
#                     break
#             if verbose:
#                 print(fringe)

#     # If player is the horizontal
#     elif player == PLAYER_O:
#         for index, symbol in enumerate([layer[0] for layer in board]):
#             if symbol == PLAYER_O:
#                 fringe.append((index,0,1))
#         if verbose:
#             print(fringe)
#         while True:
#             # If we can no longer expand return 0
#             if len(fringe) == 0:
#                 if verbose:
#                     print(visited, max_depth, BOARD_SIZE)
#                 return max_depth/BOARD_SIZE
#             top = fringe[0]
#             row,col,depth = top
#             # If the level is at the other side
#             if col == BOARD_SIZE -1:
#                 if verbose:
#                     print(visited)
#                 return 1
#             # Adding the expanded element to the visited set
#             visited.add(top)
#             # Remove the expanded element from the list
#             fringe = fringe[1:]
#             # Add child nodes to the fringe
#             # If the node is top most, we start checking at the direct rightward element
#             # If the node is not top most, we start checking at the element upward of it
#             new_row = row if row == 0 else row-1
#             new_col = col + 1
#             new_depth = depth + 1
#             while True:
#                 if board[new_row][new_col] == PLAYER_O:
#                     current = (new_row,new_col,new_depth)
#                     if current not in visited:
#                         # if the node that is connected is never visited before push it onto the fringe
#                         # print(current)
#                         fringe = [current] + fringe
#                         if new_depth > max_depth:
#                             max_depth = new_depth
#                 else:
#                     # check if the element is not the top most element
#                     if new_row != row - 1: 
#                         break
#                 new_row += 1
#                 if new_row == BOARD_SIZE:
#                     break
#             if verbose:
#                 print(fringe)

# def evaluate(board,verbose=False):
#     if verbose:
#         print("X:")
#     x = path_find(board,PLAYER_X,verbose)
#     if x == 1:
#         return 1
#     if verbose:
#         print("O:")
#     o = path_find(board,PLAYER_O,verbose)
#     if o == 1:
#         return -1
#     return x - o

# # def convert_board(string):
# #     return [list(line) for line in string.split("\n")]

# # BOARD_SIZE = 3

# # sx = """XX \nOO \n   """
# # bx = convert_board(sx)


# # print(path_find(bx,PLAYER_X,True))
# # print()
# # print(path_find(bx,PLAYER_O,True))
# # print()
# # print(evaluate(bx,True))


In [91]:
def path_find(board, player, verbose=False):
    # return 1 if the player has a path else return the fractional path
    fringe = []
    visited = set()
    max_depth = 0
    # If player is the vertical
    if player == PLAYER_X:
        # add every piece to the fringe and setting their depth to 1
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if board[i][j] == PLAYER_X:
                    fringe.append((i,j,1))
                    max_depth = 1
        if verbose:
            print(fringe)
        while True:
            # If we can no longer expand return the progress
            if len(fringe) == 0:
                if verbose:
                    # print("empty fringe")
                    print(visited, max_depth, BOARD_SIZE)
                return max_depth/BOARD_SIZE
            top = fringe[0]
            row,col,depth = top
            # If the level is at the other side
            if max_depth == BOARD_SIZE:
                if verbose:
                    # print("reached")
                    print(visited)
                return 1
            # Adding the expanded element to the visited set
            visited.add((row,col))
            # Remove the expanded element from the list
            fringe = fringe[1:]
            # Add child nodes to the fringe
            # check the left node and the right node of under and above
            # top right
            new_col = col
            new_row = row - 1
            new_depth = max(depth - 1,1)
            if new_row >= 0:
                while True:
                    if board[new_row][new_col] == PLAYER_X:
                        current = (new_row,new_col,new_depth)
                        if (new_row,new_col) not in visited:
                            if verbose:
                                print(f"top right:{current}")
                            # if the node that is connected is never visited before push it onto the fringe
                            fringe = [current] + fringe
                            if new_depth > max_depth:
                                max_depth = new_depth
                    else:
                        break
                    new_col += 1
                    if new_col >= BOARD_SIZE:
                        break
            # top left
            new_col = col - 1
            new_row = row - 1
            new_depth = max(depth - 1,1)
            if new_row >= 0 and new_col >= 0:
                while True:
                    if board[new_row][new_col] == PLAYER_X:
                        current = (new_row,new_col,new_depth)
                        if (new_row,new_col) not in visited:
                            if verbose:
                                print(f"top left:{current}")
                            # if the node that is connected is never visited before push it onto the fringe
                            fringe = [current] + fringe
                            if new_depth > max_depth:
                                max_depth = new_depth
                    else:
                        break
                    new_col -= 1
                    if new_col < 0:
                        break
            # bottom right
            new_col = col
            new_row = row + 1
            new_depth = depth + 1
            if new_row < BOARD_SIZE:
                while True:
                    if board[new_row][new_col] == PLAYER_X:
                        current = (new_row,new_col,new_depth)
                        if (new_row,new_col) not in visited:
                            if verbose:
                                print(f"bottom right:{current}")
                            # if the node that is connected is never visited before push it onto the fringe
                            fringe = [current] + fringe
                            if new_depth > max_depth:
                                max_depth = new_depth
                    else:
                        break
                    new_col += 1
                    if new_col >= BOARD_SIZE:
                        break
            # bottom left
            new_col = col - 1
            new_row = row + 1
            new_depth = depth + 1
            if new_col >=0 and new_row < BOARD_SIZE:
                while True:
                    if board[new_row][new_col] == PLAYER_X:
                        current = (new_row,new_col,new_depth)
                        if (new_row,new_col) not in visited:
                            if verbose:
                                print(f"bottom left:{current}")
                            # if the node that is connected is never visited before push it onto the fringe
                            fringe = [current] + fringe
                            if new_depth > max_depth:
                                max_depth = new_depth
                    else:
                        break
                    new_col -= 1
                    if new_col < 0:
                        break

            if verbose:
                print(fringe)

    # If player is the horizontal
    elif player == PLAYER_O:
        # add every piece to the fringe and setting their depth to 1
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if board[j][i] == PLAYER_O:
                    fringe.append((j,i,1))
                    max_depth = 1
        if verbose:
            print(fringe)
        while True:
            # If we can no longer expand return the progress
            if len(fringe) == 0:
                if verbose:
                    print(visited, max_depth, BOARD_SIZE)
                return max_depth/BOARD_SIZE
            top = fringe[0]
            row,col,depth = top
            # If the level is at the other side
            if verbose:
                print(f"top = {top} fringe = {fringe}")
            if max_depth == BOARD_SIZE:
                if verbose:
                    print(visited)
                return 1
            # Adding the expanded element to the visited set
            visited.add((row,col))
            # Remove the expanded element from the list
            fringe = fringe[1:]
            # Add child nodes to the fringe
            # check left-up, left-down, right-up and right-down
            # left up
            new_col = col - 1
            new_row = row
            new_depth = max(depth - 1,1)
            if new_col >= 0:
                while True:
                    # print(new_row,new_col)
                    if board[new_row][new_col] == PLAYER_O:
                        current = (new_row,new_col,new_depth)
                        if (new_row,new_col) not in visited:
                            if verbose:
                                print(f"left up:{current}")
                            # if the node that is connected is never visited before push it onto the fringe
                            fringe = [current] + fringe
                            if new_depth > max_depth:
                                max_depth = new_depth
                    else:
                        break
                    new_row -= 1
                    if new_row < 0:
                        break
            # left down
            new_col = col - 1
            new_row = row + 1
            new_depth = max(depth - 1,1)
            if new_col >= 0 and new_row < BOARD_SIZE:
                while True:
                    if board[new_row][new_col] == PLAYER_O:
                        current = (new_row,new_col,new_depth)
                        if (new_row,new_col) not in visited:
                            if verbose:
                                print(f"left down:{current}")
                            # if the node that is connected is never visited before push it onto the fringe
                            fringe = [current] + fringe
                            if new_depth > max_depth:
                                max_depth = new_depth
            
                    else:
                        break
                    new_row += 1
                    if new_row >= BOARD_SIZE:
                        break
            # right down
            new_col = col + 1
            new_row = row
            new_depth = depth + 1
            if new_col < BOARD_SIZE:
                while True:
                    if board[new_row][new_col] == PLAYER_O:
                        current = (new_row,new_col,new_depth)
                        if (new_row,new_col) not in visited:
                            if verbose:
                                print(f"right down:{current}")
                            # if the node that is connected is never visited before push it onto the fringe
                            fringe = [current] + fringe
                            if new_depth > max_depth:
                                max_depth = new_depth
                    else:
                        break
                    new_row += 1
                    if new_row >= BOARD_SIZE:
                        break
            # right up
            new_col = col + 1
            new_row = row - 1
            new_depth = depth + 1
            if new_row >=0 and new_col < BOARD_SIZE:
                while True:
                    if board[new_row][new_col] == PLAYER_O:
                        current = (new_row,new_col,new_depth)
                        if (new_row,new_col) not in visited:
                            if verbose:
                                print(f"right up:{current}")
                            # if the node that is connected is never visited before push it onto the fringe
                            fringe = [current] + fringe
                            if new_depth > max_depth:
                                max_depth = new_depth
                    else:
                        break
                    new_row -= 1
                    if new_row < 0:
                        break

# a function that returns 1 or -1 on win conditions
# and returns the difference between the two players' max depth if no one is winning
def evaluate(board,verbose=False):
    if verbose:
        print("X:")
    x = path_find(board,PLAYER_X,verbose)
    if x == 1:
        if verbose:
            print("evaluate: Xwins")
        return 1
    if verbose:
        print("O:")
    o = path_find(board,PLAYER_O,verbose)
    if o == 1:
        if verbose:
            print("evaluate: Owins")
        return -1
    if verbose:
        print(f"evaluate: x = {x} o = {o}")
    return x - o

This code block is used to test the path finding algorithm on manually created boards

In [92]:
# def convert_board(string):
#     return [list(line) for line in string.split("\n")]

# BOARD_SIZE = 4

# # \X X O X\
# #  \O X X O\
# #   \X O O X\
# #    \O   O  \

# board = convert_board("XXOX\nOXXO\nXOOX\nO O ")
# print(path_find(board,PLAYER_O,True))
# evaluate(board)

## Minimax function

A function that does minimaxing on every child possible of the board state given

This function comes with depth limiting as well as alpha beta pruning

Although it is depth limited, we can not limit the depth by setting depth = inf

In [93]:
# Define the minimax function with alpha-beta pruning
def minimax(board, depth, alpha, beta, maximizing_player):
    result = evaluate(board)
    if result in [-1,1] or depth == 0 or isfull(board):
        return result
    
    if maximizing_player:
        max_eval = -math.inf
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if board[i][j] == EMPTY:
                    board[i][j] = PLAYER_X
                    # print(f"minimax: {depth} {i} {j}")
                    eval = minimax(board, depth - 1, alpha, beta, False)
                    board[i][j] = EMPTY
                    if eval == 1:
                        return 1
                    max_eval = max(max_eval, eval)
                    alpha = max(alpha, eval)
                    if beta <= alpha:
                        break
        return max_eval
    else:
        min_eval = math.inf
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if board[i][j] == EMPTY:
                    board[i][j] = PLAYER_O
                    # print(f"minimax: {depth} {i} {j}")
                    eval = minimax(board, depth - 1, alpha, beta, True)
                    board[i][j] = EMPTY
                    if eval == -1:
                        return -1
                    min_eval = min(min_eval, eval)
                    beta = min(beta, eval)
                    if beta <= alpha:
                        break
        return min_eval

## Get best move

Iterate through all the possible move at this stage, evaluating by minimax and returns the best move

The iterative version is a special version where the initial depth of the minimax can be parsed in

In [148]:
def get_best_move(board, player, verbose=False):
    best_move = None
    best_eval = 0
    if player == PLAYER_X:
        best_eval = -math.inf
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if board[i][j] == EMPTY:
                    board[i][j] = PLAYER_X
                    # print("best_move",i,j)
                    eval = minimax(board, DEPTH, -math.inf, math.inf, False)
                    if verbose:
                        print(f"Move ({i},{j}) eval = {eval}")
                    board[i][j] = EMPTY
                    if eval == 1:
                        best_eval = 1
                        best_move = (i, j)
                        break
                    if eval > best_eval:
                        best_eval = eval
                        best_move = (i, j)
    elif player == PLAYER_O:
        best_eval = math.inf
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if board[i][j] == EMPTY:
                    board[i][j] = PLAYER_O
                    # print("best_move",i,j)
                    eval = minimax(board, DEPTH, -math.inf, math.inf, True)
                    if verbose:
                        print(f"Move ({i},{j}) eval = {eval}")
                    board[i][j] = EMPTY
                    if eval == -1:
                        best_eval = -1
                        best_move = (i, j)
                        break
                    if eval < best_eval:
                        best_eval = eval
                        best_move = (i, j)
    if best_move == None:
        return (-1,-1)
    print(f"Best move for {player}: {best_move} of score {best_eval}")
    return best_move

# add a way to give depth to the get_best_move
def get_best_move_iterative(board, player, depth, verbose=False):
    best_move = None
    best_eval = 0
    if player == PLAYER_X:
        best_eval = -math.inf
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if board[i][j] == EMPTY:
                    board[i][j] = PLAYER_X
                    # print("best_move",i,j)
                    eval = minimax(board, depth, -math.inf, math.inf, False)
                    if verbose:
                        print(f"Move ({i},{j}) eval = {eval}")
                    board[i][j] = EMPTY
                    if eval == 1:
                        best_eval = 1
                        best_move = (i, j)
                        break
                    if eval > best_eval:
                        best_eval = eval
                        best_move = (i, j)
    elif player == PLAYER_O:
        best_eval = math.inf
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if board[i][j] == EMPTY:
                    board[i][j] = PLAYER_O
                    # print("best_move",i,j)
                    eval = minimax(board, depth, -math.inf, math.inf, True)
                    if verbose:
                        print(f"Move ({i},{j}) eval = {eval}")
                    board[i][j] = EMPTY
                    if eval == -1:
                        best_eval = -1
                        best_move = (i, j)
                        break
                    if eval < best_eval:
                        best_eval = eval
                        best_move = (i, j)
    if best_move == None:
        return (-1,-1)
    print(f"Best move for {player}: {best_move} of score {best_eval}")
    return best_move

## Main functions

a game loop that creates an empty board and simulating bots by getting the best moves alternatingly

the iterative version allows for iterative deepening of the minimax search tree

In [149]:
def main(verbose=False):
    # Initialize the empty board
    board = [[EMPTY for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
    # Print the initial board
    print("Initial Board:")
    for index, row in enumerate(board):
        print(index*" "+ "\\" + " ".join(["_" if elm == EMPTY else elm for elm in row]) + "\\")
    print()
    # Define the current player (start with Player X)
    current_player = PLAYER_X
    # Game loop
    while True:
        # Get the best move for the current player
        x, y = get_best_move(board, current_player, verbose)
        if x == -1 and y == -1:
            if current_player == PLAYER_X:
                print(f"Player O wins!")
                break
            else:
                print(f"Player X wins!")
                break
        # Make the move on the board
        board[x][y] = current_player
        print(f"Player {current_player}'s move:")
        for index, row in enumerate(board):
            print(index*" "+ "\\" + " ".join(["_" if elm == EMPTY else elm for elm in row]) + "\\")
        print()
        # Check for win or draw
        result = evaluate(board)
        if result == 1:
            print(f"Player X wins!")
            break
        elif result == -1:
            print(f"Player O wins!")
            break
        current_player = PLAYER_O if current_player == PLAYER_X else PLAYER_X

def main_iterative(max_depth=inf, progression=1, verbose=False):
    # Initialize the empty board
    board = [[EMPTY for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
    # Print the initial board
    print("Initial Board:")
    for index, row in enumerate(board):
        print(index*" "+ "\\" + " ".join(["_" if elm == EMPTY else elm for elm in row]) + "\\")
    print()
    # Define the current player (start with Player X)
    current_player = PLAYER_X
    # Progressive depth
    depth = 0
    # Game loop
    while True:
        # Get the best move for the current player and iterate the depth
        x, y = get_best_move_iterative(board, current_player, depth, verbose)
        depth = min(max_depth, round(depth + progression))
        # Make the move on the board
        board[x][y] = current_player
        print(f"Player {current_player}'s move:")
        for index, row in enumerate(board):
            print(index*" "+ "\\" + " ".join(["_" if elm == EMPTY else elm for elm in row]) + "\\")
        print()
        # Check for win or draw
        result = evaluate(board)
        if result == 1:
            print(f"Player X wins!")
            break
        elif result == -1:
            print(f"Player O wins!")
            break
        current_player = PLAYER_O if current_player == PLAYER_X else PLAYER_X

## Ex1: full depth

(only works in feasible time with 3x3 board or smaller)

In [150]:
BOARD_SIZE = 3
DEPTH = inf

main(True)

Initial Board:
\_ _ _\
 \_ _ _\
  \_ _ _\

Move (0,0) eval = -1
Move (0,1) eval = -1
Move (0,2) eval = 1
Move (1,0) eval = 1
Move (2,0) eval = 1
Best move for X: (2, 0) of score 1
Player X's move:
\_ _ _\
 \_ _ _\
  \X _ _\

Move (0,0) eval = 1
Move (0,1) eval = 1
Move (0,2) eval = 1
Move (1,0) eval = 1
Move (1,1) eval = 1
Move (1,2) eval = 1
Move (2,1) eval = 1
Move (2,2) eval = 1
Best move for O: (0, 0) of score 1
Player O's move:
\O _ _\
 \_ _ _\
  \X _ _\

Move (0,1) eval = 1
Move (1,0) eval = 1
Move (2,1) eval = -1
Move (2,2) eval = -1
Best move for X: (1, 0) of score 1
Player X's move:
\O _ _\
 \X _ _\
  \X _ _\

Move (0,1) eval = 1
Move (0,2) eval = 1
Move (1,1) eval = 1
Move (1,2) eval = 1
Move (2,1) eval = 1
Move (2,2) eval = 1
Best move for O: (0, 1) of score 1
Player O's move:
\O O _\
 \X _ _\
  \X _ _\

Move (0,2) eval = 1
Move (1,1) eval = -1
Move (1,2) eval = -1
Move (2,1) eval = -1
Move (2,2) eval = -1
Best move for X: (0, 2) of score 1
Player X's move:
\O O X\
 \X _ _\


## Ex2: limited depth

Works well with medium sized boards and medium to low depth

In [151]:
BOARD_SIZE = 4

main_iterative(1, 5, True)

Initial Board:
\_ _ _ _\
 \_ _ _ _\
  \_ _ _ _\
   \_ _ _ _\

Move (0,0) eval = 0.25
Move (0,1) eval = 0.25
Move (0,2) eval = 0.25
Move (0,3) eval = 0.25
Move (1,0) eval = 0.25
Move (1,1) eval = 0.25
Move (1,2) eval = 0.25
Move (1,3) eval = 0.25
Move (2,0) eval = 0.25
Move (2,1) eval = 0.25
Move (2,2) eval = 0.25
Move (2,3) eval = 0.25
Move (3,0) eval = 0.25
Move (3,1) eval = 0.25
Move (3,2) eval = 0.25
Move (3,3) eval = 0.25
Best move for X: (0, 0) of score 0.25
Player X's move:
\X _ _ _\
 \_ _ _ _\
  \_ _ _ _\
   \_ _ _ _\

Move (0,1) eval = 0.25
Move (0,2) eval = 0.25
Move (0,3) eval = 0.25
Move (1,0) eval = 0.0
Move (1,1) eval = 0.25
Move (1,2) eval = 0.25
Move (1,3) eval = 0.25
Move (2,0) eval = 0.25
Move (2,1) eval = 0.25
Move (2,2) eval = 0.25
Move (2,3) eval = 0.25
Move (3,0) eval = 0.25
Move (3,1) eval = 0.25
Move (3,2) eval = 0.25
Move (3,3) eval = 0.25
Best move for O: (1, 0) of score 0.0
Player O's move:
\X _ _ _\
 \O _ _ _\
  \_ _ _ _\
   \_ _ _ _\

Move (0,1) eval = -0.25

Move (1,1) eval = 0.25
Move (1,2) eval = 0.25
Move (1,3) eval = 0.25
Move (2,0) eval = 0.25
Move (2,1) eval = 0.0
Move (2,2) eval = 0.25
Move (2,3) eval = 0.25
Move (3,0) eval = 0.25
Move (3,1) eval = 0.25
Move (3,2) eval = 0.25
Move (3,3) eval = 0.25
Best move for O: (2, 1) of score 0.0
Player O's move:
\X _ X _\
 \O _ _ _\
  \_ O _ _\
   \_ _ _ _\

Move (0,1) eval = -0.25
Move (0,3) eval = -0.25
Move (1,1) eval = -0.25
Move (1,2) eval = -0.25
Move (1,3) eval = 0.0
Move (2,0) eval = -0.25
Move (2,2) eval = 0.0
Move (2,3) eval = 0.0
Move (3,0) eval = -0.25
Move (3,1) eval = -0.25
Move (3,2) eval = -0.25
Move (3,3) eval = -0.25
Best move for X: (1, 3) of score 0.0
Player X's move:
\X _ X _\
 \O _ _ X\
  \_ O _ _\
   \_ _ _ _\

Move (0,1) eval = 1
Move (0,3) eval = 0.25
Move (1,1) eval = 1
Move (1,2) eval = 0.25
Move (2,0) eval = 1
Move (2,2) eval = 1
Move (2,3) eval = 1
Move (3,0) eval = 1
Move (3,1) eval = 1
Move (3,2) eval = 0.0
Move (3,3) eval = 1
Best move for O: (3, 2) of score 0.0

## Ex3: Iterative depth

Works great with medium sized boards but fall of with larger boards

In [152]:
BOARD_SIZE = 4

main_iterative(1, inf, True)

Initial Board:
\_ _ _ _\
 \_ _ _ _\
  \_ _ _ _\
   \_ _ _ _\

Move (0,0) eval = 0.25
Move (0,1) eval = 0.25
Move (0,2) eval = 0.25
Move (0,3) eval = 0.25
Move (1,0) eval = 0.25
Move (1,1) eval = 0.25
Move (1,2) eval = 0.25
Move (1,3) eval = 0.25
Move (2,0) eval = 0.25
Move (2,1) eval = 0.25
Move (2,2) eval = 0.25
Move (2,3) eval = 0.25
Move (3,0) eval = 0.25
Move (3,1) eval = 0.25
Move (3,2) eval = 0.25
Move (3,3) eval = 0.25
Best move for X: (0, 0) of score 0.25
Player X's move:
\X _ _ _\
 \_ _ _ _\
  \_ _ _ _\
   \_ _ _ _\

Move (0,1) eval = 0.25
Move (0,2) eval = 0.25
Move (0,3) eval = 0.25
Move (1,0) eval = 0.0
Move (1,1) eval = 0.25
Move (1,2) eval = 0.25
Move (1,3) eval = 0.25
Move (2,0) eval = 0.25
Move (2,1) eval = 0.25
Move (2,2) eval = 0.25
Move (2,3) eval = 0.25
Move (3,0) eval = 0.25
Move (3,1) eval = 0.25
Move (3,2) eval = 0.25
Move (3,3) eval = 0.25
Best move for O: (1, 0) of score 0.0
Player O's move:
\X _ _ _\
 \O _ _ _\
  \_ _ _ _\
   \_ _ _ _\

Move (0,1) eval = -0.25

# BACKUP PROJECT: Dots and Boxes

This game AI was made by myself in fear that the HEX game's performance will not be sufficient for this project,<br> however Mr. Paulo had clarify that my HEX is alright, since then this project is since abandoned.<br> I left the code here so that you can see my approach to solving problems when it comes to a different game.

## Game functions

In [153]:
import copy
import math

In [154]:
BOARD_SIZE = 3

# undrawn line or box
EMPTY = " "
# mark of player 1
PLAYER_X = "X"
# mark of player 2
PLAYER_O = "O"

In [155]:
class DotsandBoxes:

    def __init__(self, board_size=BOARD_SIZE):
        self.board_size = board_size
        # a 2D array of all the squares drawn/to be drawn
        self.sqs = [[EMPTY for col in range(board_size)] for row in range(board_size)]
        # a 2D array of all the horizontal lines drawn/to be drawn
        self.hor = [[EMPTY for col in range(board_size)] for row in range(board_size+1)]
        # a 2D array of all the vertical lines drawn/to be drawn
        self.ver = [[EMPTY for col in range(board_size+1)] for row in range(board_size)]

    def __str__(self):
        s = ""
        s += f"Sqs: {str(self.sqs)}\n"
        s += f"Hor: {str(self.hor)}\n"
        s += f"Ver: {str(self.ver)}\n"
        return s
    
    def copy(self):
        return copy.deepcopy(self)

    def is_over(self):
        for row in self.sqs:
            for elm in row:
                if elm == EMPTY:
                    return False
        return True

    def count_scores(self):
        x_count = 0
        o_count = 0
        for row in self.sqs:
            for elm in row:
                if elm == PLAYER_X:
                    x_count += 1
                elif elm == PLAYER_O:
                    o_count += 1
        return x_count, o_count
    
    def evaluate(self):
        x_count, o_count = self.count_scores()
        diff = x_count - o_count
        # If x wins give really high score
        if self.is_over() and diff > 0:
            return self.board_size**2 + 1
        # If y wins give really low score
        elif self.is_over() and diff < 0:
            return -self.board_size**2 - 1
        # if not return the square counts differences
        return diff
    
    def get_availible_moves(self):
        # check to see which lines are not drawn
        availible_moves = set()
        for i in range(self.board_size):
            for j in range(self.board_size):
                if self.hor[i][j] == EMPTY:
                    availible_moves.add((i,j,"hor"))
                if self.ver[i][j] == EMPTY:
                    availible_moves.add((i,j,"ver"))
        for k in range(self.board_size):
            if self.hor[self.board_size][k] == EMPTY:
                availible_moves.add((self.board_size+1,k,"hor"))
            if self.ver[k][self.board_size] == EMPTY:
                availible_moves.add((k,self.board_size+1,"ver"))
        return availible_moves

    def check(self,row,col,edge):
        # check if horizontal
        squares = set()
        if edge == "hor":
            # check if it is not the bottom most -> check the square below it
            if row != self.board_size and self.hor[row+1][col] != EMPTY and self.ver[row][col] != EMPTY and self.ver[row][col+1] != EMPTY:
                squares.add((row,col))
            # check if it is not the top most -> check the square above it  
            if row != 0 and self.hor[row-1][col] != EMPTY and self.ver[row-1][col] != EMPTY and self.ver[row-1][col+1] != EMPTY:
                squares.add((row-1,col))
            # if not no square
        elif edge == "ver":
            # check if it is not the right most -> check the square to the right
            if col != self.board_size and self.hor[row+1][col] != EMPTY and self.hor[row][col] != EMPTY and self.ver[row][col+1] != EMPTY:
                squares.add((row,col))
            # check if it is not the left most -> check the square to the left  
            if col != 0 and self.hor[row][col-1] != EMPTY and self.hor[row+1][col-1] != EMPTY and self.ver[row][col-1] != EMPTY:
                squares.add((row,col-1))
            # if not no square
        return squares

    
    def minimax(self, depth, alpha, beta, maximizing_player):
        
        # check the end conditions: game over or max depth reached
        if self.is_over() or depth == 0:
            return self.evaluate()
        
        # if PLAYER_X
        if maximizing_player:
            # initial max = -inf
            max_eval = -math.inf
            # get all the availible moves and iterate through
            availible_moves = self.get_availible_moves()
            for move in availible_moves:
                row,col,edge = move
                # create a copy of the original class object
                dnb = self.copy()
                # check if there's any square claimable and claim them
                if edge == "hor":
                    dnb.hor[row][col] == PLAYER_X
                elif edge == "ver":
                    dnb.ver[row][col] == PLAYER_X
                square_set = dnb.check(row,col,edge)
                for square in square_set:
                    sqs_row, sqs_col = square
                    dnb.sqs[sqs_row][sqs_col] = PLAYER_X
                # do minimax on this child
                eval = dnb.minimax(depth - 1, alpha, beta, False)
                # return right away if PLAYER_X is winning
                if eval == self.board_size**2 +1:
                    return self.board_size**2 +1
                # max
                max_eval = max(max_eval, eval)
                # alpha cut
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval
        else:
            # initial min = inf
            min_eval = math.inf
            # get all the availible moves and iterate through
            availible_moves = self.get_availible_moves()
            for move in availible_moves:
                row,col,edge = move
                # create a copy of the original class object
                dnb = self.copy()
                # check if there's any square claimable and claim them
                if edge == "hor":
                    dnb.hor[row][col] == PLAYER_O
                elif edge == "ver":
                    dnb.ver[row][col] == PLAYER_O
                square_set = dnb.check(row,col,edge)
                for square in square_set:
                    sqs_row, sqs_col = square
                    dnb.sqs[sqs_row][sqs_col] = PLAYER_O
                # do minimax on this child
                eval = dnb.minimax(depth - 1, alpha, beta, True)
                # return right away if PLAYER_O is winning
                if eval == -self.board_size**2 +1:
                    return -self.board_size**2 +1
                # min
                min_eval = min(min_eval, eval)
                # beta cut
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval
        
    # def get_best_move(self, player):
    #     if player == PLAYER_X:



## Testing of the functions written

In [156]:
dnb = DotsandBoxes()

In [157]:
for i in range(3):
    for j in range(3):
        dnb.sqs[i][j] = PLAYER_X

In [158]:
dnb.count_scores()

(9, 0)

In [159]:
dnb.is_over()

True

In [160]:
dnb2 = copy.deepcopy(dnb)

In [161]:
dnb2.sqs[0][0] = PLAYER_O

In [162]:
len(dnb.get_availible_moves())

24

In [163]:
dnb = DotsandBoxes()

dnb.hor[2][2] = PLAYER_X
dnb.hor[3][2] = PLAYER_X
dnb.ver[2][2] = PLAYER_X

dnb.check(2,3,'ver')

{(2, 2)}

In [164]:
print(dnb)

Sqs: [[' ', ' ', ' '], [' ', ' ', ' '], [' ', ' ', ' ']]
Hor: [[' ', ' ', ' '], [' ', ' ', ' '], [' ', ' ', 'X'], [' ', ' ', 'X']]
Ver: [[' ', ' ', ' ', ' '], [' ', ' ', ' ', ' '], [' ', ' ', 'X', ' ']]



In [165]:
len(dnb.get_availible_moves())

21